In [30]:
import numpy as np
import pandas as pd
import re

pd.set_option('display.max_columns', None)

In [31]:
df = pd.read_csv("../csv-archivos/HR RAW DATA.csv", index_col=0)
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,employeecount,employeenumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear,WORKLIFEBALANCE,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YEARSWITHCURRMANAGER,SameAsMonthlyIncome,DateBirth,Salary,RoleDepartament,NUMBERCHILDREN,RemoteWork
0,51,No,NaN,"684,0$",NaN,6,3,NaN,1,"162,0",1,0,51,3,5,resEArch DIREcToR,3,NaN,"19537,0",6462,7,Y,No,13,"3,0",3,NaN,0,NaN,5,"3,0",20,NaN,15,15,"19537,0",1972,1000000000$,NaN,NaN,Yes
1,52,No,NaN,"699,0$",NaN,1,4,Life Sciences,1,"259,0",3,0,65,2,5,ManAGeR,3,NaN,"19999,0",5678,0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,"19999,0",1971,1000000000$,NaN,NaN,1
2,42,No,travel_rarely,"532,0$",Research & Development,4,2,Technical Degree,1,"319,0",3,0,58,3,5,ManaGER,4,Married,"19232,0",4933,1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,"19232,0",1981,1000000000$,ManaGER - Research & Development,NaN,1
3,47,No,travel_rarely,"359,0$",NaN,2,4,Medical,1,NaN,1,1,82,3,4,ReseArCH DIrECtOr,3,Married,"17169,0",26703,3,Y,NaN,19,"3,0",2,NaN,2,NaN,2,NaN,20,NaN,5,6,"17169,0",1976,1000000000$,NaN,NaN,False
4,46,No,NaN,"1319,0$",NaN,3,3,Technical Degree,1,NaN,1,1,45,4,4,sAleS EXECUtIve,1,Divorced,NaN,7739,2,Y,No,12,"3,0",4,NaN,1,NaN,5,"3,0",19,NaN,2,8,NaN,1977,1000000000$,NaN,NaN,0


# FUNCIONES

In [32]:
def word_to_num(dato):
    
    if len(dato) > 2:
        dato = w2n.word_to_num(dato)
        return dato

    else:
        return dato

In [33]:
def true_false(valor):
   
    if valor in ['Yes', 'yes', '1', 'Y']:
        valor = True
    
    elif valor in ['False', 'No', '0', 'false']:
        valor = False
    

    else:
        valor = np.nan

    return valor


In [34]:
def to_float(valor):
    try:
        if type(valor) != str or np.nan:
            str(valor)
        
            
        if '$' in valor:
            valor = valor.replace('$', '')
        elif '%' in valor:
            valor = valor.replace('%', '')
        return float(valor.replace(",", "."))
        
    
    except:
        
        return np.nan

In [35]:
def unificacion(valor):
    if type(valor) == str:
        return valor.lower().strip()
    else:
        return valor

# LISTAS

In [36]:
lista_bool = ['Attrition', 'RemoteWork', 'OverTime']
lista_float =  ['employeenumber', 'PerformanceRating', 'TOTALWORKINGYEARS', 'YearsInCurrentRole', 'DailyRate', 'WORKLIFEBALANCE', 'MonthlyIncome']
# lista_object = df.describe(include='object')

# LIMPIEZA

In [37]:
df.drop(['employeecount', 'NUMBERCHILDREN', 'SameAsMonthlyIncome', 'Salary', 'Age', 'Over18', 'StandardHours', 'RoleDepartament', 'YearsInCurrentRole'], axis=1, inplace=True)

In [ ]:
df['environment_satisfaction'] = df['environment_satisfaction'].apply(lambda x: x if x in range(1,5) else np.nan)

In [38]:
df['BusinessTravel'] = df['BusinessTravel'].map({"non-travel":"no", "travel_rarely":"rarely", "travel_frequently":"frequently"})

In [39]:
df["DistanceFromHome"] = abs(df["DistanceFromHome"])

In [40]:
df['Gender'] = df['Gender'].map({1: "F", 0: 'M'})

In [41]:
df['MaritalStatus'] = df['MaritalStatus'].str.replace('marreid', 'married')


In [42]:
df['HourlyRate'] = df['HourlyRate'].replace('Not Available', np.nan)

In [43]:
df["PercentSalaryHike"] = (df["PercentSalaryHike"]).astype(float) / 100

In [44]:
for columna in lista_float:

    df[columna] = df[columna].apply(to_float)

In [45]:
for columna in lista_bool:
    df[columna] = df[columna].apply(true_false)

In [46]:
df = df.apply(unificacion)

In [47]:
nuevas_columnas = []
for nombre in df.columns:

    patron = re.compile(r'[a-zA-Z][a-z]*[A-Z][a-z][\d]*')

    if re.search(patron, nombre):
        col_sep = re.sub(r'(?<=[a-z])(?=[A-Z])', '_', nombre) #cada vez que encuentro una minúscula seguida de mayúscula lo cambio por _ 
        nuevas_columnas.append(col_sep.lower().strip())
    
    elif nombre == 'employeenumber':
        nuevas_columnas.append('employee_number')
    
    elif nombre == 'NUMCOMPANIESWORKED':
        nuevas_columnas.append('num_companies_worked')
    
    elif nombre == 'TOTALWORKINGYEARS':
        nuevas_columnas.append('total_working_years')

    elif nombre == 'WORKLIFEBALANCE':
            nuevas_columnas.append('work_life_balance')

    elif nombre == 'YEARSWITHCURRMANAGER':
        nuevas_columnas.append('years_with_current_manager')
    
    else:
        nuevas_columnas.append(nombre.lower())
    

    dict_columns = {}

    for indice, valor in zip(df.columns, nuevas_columnas):
        dict_columns[indice] = valor

    df.rename(columns=dict_columns, inplace=True)



In [48]:
lista_object = df.describe(include='object')

In [49]:
for columna in lista_object:
    df[columna] = df[columna].apply(unificacion)

In [52]:
df.drop_duplicates(inplace=True)

In [53]:
df

,attrition,business_travel,daily_rate,department,distance_from_home,education,education_field,employee_number,environment_satisfaction,gender,hourly_rate,job_involvement,job_level,job_role,job_satisfaction,marital_status,monthly_income,monthly_rate,num_companies_worked,over_time,percent_salary_hike,performance_rating,relationship_satisfaction,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_in_current_role,years_since_last_promotion,years_with_current_manager,date_birth,remote_work
0,False,NaN,684.0,NaN,6,3,NaN,162.0,1,m,51,3,5,research director,3,NaN,19537.0,6462,7,False,0.13,3.0,3,0,NaN,5,3.0,20,NaN,15,15,1972,True
1,False,NaN,699.0,NaN,1,4,life sciences,259.0,3,m,65,2,5,manager,3,NaN,19999.0,5678,0,NaN,0.14,3.0,1,1,34.0,5,3.0,33,NaN,11,9,1971,True
2,False,rarely,532.0,research & development,4,2,technical degree,319.0,3,m,58,3,5,manager,4,married,19232.0,4933,1,False,0.11,3.0,4,0,22.0,3,NaN,22,NaN,11,15,1981,True
3,False,rarely,359.0,NaN,2,4,medical,NaN,1,f,82,3,4,research director,3,married,17169.0,26703,3,NaN,0.19,3.0,2,2,NaN,2,NaN,20,NaN,5,6,1976,False
4,False,NaN,1319.0,NaN,3,3,technical degree,NaN,1,f,45,4,4,sales executive,1,divorced,NaN,7739,2,False,0.12,3.0,4,1,NaN,5,3.0,19,NaN,2,8,1977,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,False,NaN,498.0,NaN,3,4,medical,966.0,3,f,93,3,2,manufacturing director,1,married,6725.0,13554,1,NaN,0.12,3.0,3,1,8.0,2,4.0,8,NaN,6,3,1991,True
1610,False,no,805.0,NaN,4,2,NaN,972.0,3,m,57,3,2,laboratory technician,2,NaN,4447.0,23163,1,NaN,0.12,3.0,2,0,NaN,5,2.0,9,NaN,0,8,1978,True
1611,False,rarely,903.0,NaN,13,5,NaN,NaN,13,m,41,4,3,sales executive,3,single,NaN,2560,0,False,0.18,3.0,4,0,9.0,3,3.0,8,NaN,0,7,1984,True
1612,False,no,1229.0,NaN,8,4,technical degree,990.0,1,m,84,3,2,sales executive,4,divorced,NaN,25952,4,False,0.13,NaN,4,2,12.0,3,3.0,7,NaN,0,7,1987,NaN
